# Parte 1 - Extrair Dados

In [1]:
import pandas as pd
import requests

### Lista de UF's

In [2]:
def get_uf_list():
    response = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados')

    if response.status_code == 200:
        uf_metadata = response.json()
        uf_siglas = [ uf['sigla'] for uf in uf_metadata ]
        
        return uf_siglas
    else:
        raise Exception('Não foi possível obter os dados dos estados brasileiros.')

### Área de Cada Estado

In [3]:
def get_metadados_estado(uf:str):
    url = f'http://servicodados.ibge.gov.br/api/v3/malhas/estados/{uf}/metadados'

    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [4]:
def get_uf_area(lista_ufs_sligas:list):
    uf_area = {}

    for uf in lista_ufs_sligas:
        metadados = get_metadados_estado(uf)

        uf_metadado_area = metadados[0]['area']['dimensao']
        uf_area[uf] = uf_metadado_area
    
    return uf_area

### Lista de Cidades por Estado

In [5]:
def get_metadados_municipio(uf:str):
    url = f'http://servicodados.ibge.gov.br/api/v1/localidades/estados/{uf}/municipios'

    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [6]:
def get_municipios(lista_ufs_sligas:list):
    ufs_municipios = {}

    for uf in lista_ufs_sligas:
        metadados = get_metadados_municipio(uf)

        ufs_municipio = []

        for municipio in metadados:
            ufs_municipio.append(municipio['nome'])

        ufs_municipios[uf] = ufs_municipio
    
    return ufs_municipios

In [7]:
lista_ufs_sligas = get_uf_list()

lista_ufs_area = get_uf_area(lista_ufs_sligas)
lista_ufs_municipios = get_municipios(lista_ufs_sligas)

# Parte 2 - Transformar Dados

### Dados de Área por UF

In [8]:
# Transformando em DataFrame e convertendo os tipos
uf_area_df = pd.DataFrame(list(lista_ufs_area.items()), columns=['UF', 'Area'])

# Convertendo os tipos de dados
uf_area_df['Area'] = uf_area_df['Area'].astype(float)

# Ordenando por área
uf_area_df_ordenada = uf_area_df.sort_values(by='Area', ascending=False)

### Lista de Municipios por UF

In [19]:
lista_ufs_municipios_df = pd.DataFrame(list(lista_ufs_municipios.items()), columns=['UF', 'Municipios'])
lista_ufs_municipios_df_exploded = lista_ufs_municipios_df.explode('Municipios')

# Parte 3 - Salvando os Dados

In [10]:
# Salvando lista de UF's em CSV
uf_area_df_ordenada.to_csv('./data/csv/uf_area.csv', index=False, sep=';', encoding='utf-8')

In [20]:
# Salvando lista de UF's em Parquet particionado por UF
lista_ufs_municipios_df_exploded.to_parquet('./data/parquet/municipios/', index=False, partition_cols=['UF'])